<a href="https://colab.research.google.com/github/gabipalacios/Python/blob/main/RiesgoZ2capa_V11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [331]:
import numpy as np
import pandas as pd
import datetime as dt
import io
from google.colab import files
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sn
from google.colab import drive
pd.options.display.float_format = '{:.2f}'.format

drive.mount('/content/drive')
origen = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_sinpagos1.csv')
marco = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_PreMarcoBCU.csv')

#Variables seleccionadas del maro:
marco = marco.add_prefix('m_')
#Variables seleccionadas de Financial y PLataforma
origen = origen.add_prefix('o_')

#Elimino campos,tratando de evitar riesgos de plataforma y campos repetidos y con inconvenientes, (analizados previamente)
origen = origen.drop(['o_monto', 'o_cuota', 'o_tasa', 'o_plazo', 'o_cargo_originacion', 'o_tiene_seguro', 'o_tiene_cargo_adm',
                      'o_clr_score_if', 'o_clr_bp_fecha', 'o_rpf_alert', 'o_cdr_tiene_quita', 'o_cdr_tiene_oca', 'o_cdr_tiene_oca.1',
                      'o_cdr_tiene_financiera', 'o_cdr_tiene_brou', 'o_cdr_tiene_banco_privado', 'o_cdr_sum_deudas_ui', 'o_cdr_periodo',
                      'o_cdr_peor_calificacion', 'o_cdr_max_contingencia_ui', 'o_Tipo_Producto', 'o_Tipo Basico', 'o_clr_fecha',
                      'o_Canal', 'o_no_of_desc', 'o_id_solicitud', 'o_DesembolsoLugar', 'o_estado_tc', 'o_estado_pa',
                      'o_estado_cliente', 'o_cargo', 'o_meses_con_pagos', 'o_dias_atraso_max', 'o_DesembolsoSubAge.',
                      'o_DesembolsoAgencia', 'o_DesembolsoAgencia.1', 'o_CuotasPagas', 'o_localidad', 'o_Atraso', 'o_Solicitud',
                      'o_Fecha', 'o_tiene_tel_fijo', 'o_tiene_recibo_sueldo', 'o_Id_Clliente'], axis=1)

#join de las dos tablas
df = pd.merge(origen, marco, left_on='o_Documento', right_on='m_Documento', how='inner')
#Defino índice documento
df = df.set_index('m_Documento')
#Quito registros que no tienen score
df = df.dropna(subset=['o_clr_score'])

#Reorganizo valores de variables con criterio lógico, para obtener mejore conclusiones, (analizados previamente)
df.loc[~df['o_dispositivo_marca'].isin(['Apple']), 'o_dispositivo_marca'] = 'Android'
df.loc[df['o_sist_op_familia'] != 'iOS', 'o_sist_op_familia'] = 'Android'
df.loc[df['o_dispositivo_marca'] != 'Apple', 'o_dispositivo_marca'] = 'Android'
df.loc[~df['o_navegador_familia'].isin(['Facebook', 'Instagram']), 'o_navegador_familia'] = 'Web'
df.loc[~df['o_departamento'].isin(['MONTEVIDEO', 'CANELONES']), 'o_departamento'] = 'RESTO'
df['o_Zero'] = np.where(df['o_Zero'] == 'Zero', 1, 0)
df['o_clr_accion'] = np.where(df['o_clr_accion'].isin(['C2', 'C1_3_3','C4','C1_2_3','C1_3_6','C1_3_1','C3']), df['o_clr_accion'], 'Varios')

print(df.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1004, 41)


In [332]:
print(marco.shape)
print(origen.shape)
print(df.shape)

(2445, 13)
(1981, 29)
(1004, 41)


In [333]:
df = df.drop(['o_Documento'], axis=1)

In [334]:
marco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445 entries, 0 to 2444
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   m_Documento           2445 non-null   int64  
 1   m_Entidades           2445 non-null   int64  
 2   m_Exposición          2445 non-null   int64  
 3   m_Deuda               2445 non-null   int64  
 4   m_Rel_DeudaPrincipal  2445 non-null   float64
 5   m_Contingencia        2445 non-null   int64  
 6   m_Var_dueda_3M        2445 non-null   float64
 7   m_Sin_Ficha           2445 non-null   int64  
 8   m_Malos               2445 non-null   int64  
 9   m_Mejora              2445 non-null   int64  
 10  m_Grupo_b             2445 non-null   object 
 11  m_Grupo_i             2445 non-null   object 
 12  m_Q_irregular_12m     2445 non-null   int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 248.4+ KB


In [335]:
#Modificar el parámetro de la tabla df['hora'] donde se muestran las horas del día desde 0 a 23,
#son tipos de datos periódicos, utilizando el cálculo trigonométrico más adecuado para introducir el parámetro en un modelo predictivo.
df['o_Hora'] = pd.to_numeric(df['o_Hora'])

# Calcular la representación sinusoidal de la hora
df['hora_sin'] = np.sin(2 * np.pi * df['o_Hora'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['o_Hora'] / 24)

# Eliminar la columna original 'hora'
df = df.drop('o_Hora', axis=1)


In [336]:
origen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1981 entries, 0 to 1980
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   o_Documento                 1981 non-null   int64  
 1   o_Día                       1981 non-null   int64  
 2   o_Semana_año                1981 non-null   int64  
 3   o_Quincena                  1981 non-null   int64  
 4   o_Hora                      1981 non-null   int64  
 5   o_Num_Dia_Sem               1981 non-null   int64  
 6   o_Zero                      1981 non-null   object 
 7   o_TEA                       1981 non-null   float64
 8   o_status_usuario            1981 non-null   object 
 9   o_status                    1981 non-null   object 
 10  o_sist_op_familia           1972 non-null   object 
 11  o_relacion_laboral          1981 non-null   object 
 12  o_plazo_hasta               1979 non-null   float64
 13  o_nivel_educativo           1981 

In [337]:
#Quito variables propas del modelo plataforma para evitar sesgos:
df = df.drop(['o_TEA', 'o_plazo_hasta', 'o_monto_maximo','o_mejor_esquema_pricing','o_ingreso_estimado','o_Cuotas','o_Capital'], axis=1)

In [338]:
df.columns

Index(['o_Día', 'o_Semana_año', 'o_Quincena', 'o_Num_Dia_Sem', 'o_Zero',
       'o_status_usuario', 'o_status', 'o_sist_op_familia',
       'o_relacion_laboral', 'o_nivel_educativo', 'o_nivel_antiguedad_laboral',
       'o_navegador_familia', 'o_ingreso_declarado', 'o_genero', 'o_edad',
       'o_dispositivo_marca', 'o_departamento', 'o_clr_score', 'o_clr_accion',
       'o_cel_carrier', 'm_Entidades', 'm_Exposición', 'm_Deuda',
       'm_Rel_DeudaPrincipal', 'm_Contingencia', 'm_Var_dueda_3M',
       'm_Sin_Ficha', 'm_Malos', 'm_Mejora', 'm_Grupo_b', 'm_Grupo_i',
       'm_Q_irregular_12m', 'hora_sin', 'hora_cos'],
      dtype='object')

In [339]:
# prompt: cambiar los siguientes campos: 'o_Día', 'o_Semana_año', 'o_Quincena', 'o_Num_Dia_Sem', 'o_Zero','m_Contingencia' a categorico

columns_to_categorize = ['o_Día', 'o_Semana_año', 'o_Quincena', 'o_Num_Dia_Sem', 'o_Zero','m_Contingencia']

for col in columns_to_categorize:
  if col in df.columns:
    df[col] = df[col].astype('category')
  else:
    print(f"Warning: Column '{col}' not found in DataFrame.")

In [340]:
df_cat = df.select_dtypes(include=['object'])

# Transform all categorical variables into 'category' dtype
for col in df_cat.columns:
  df[col] = df[col].astype('category')

#df.info()

In [341]:
df_cat = df.select_dtypes(include=['category'])
display(df_cat)

,o_Día,o_Semana_año,o_Quincena,o_Num_Dia_Sem,o_Zero,o_status_usuario,o_status,o_sist_op_familia,o_relacion_laboral,o_nivel_educativo,o_navegador_familia,o_genero,o_dispositivo_marca,o_departamento,o_clr_accion,o_cel_carrier,m_Contingencia,m_Grupo_b,m_Grupo_i
m_Documento,,,,,,,,,,,,,,,,,,,
38753940,11,50,1,4,0,bureau_validated,validated_contact,Android,indep_sin_local,Primaria,Instagram,femenino,Android,RESTO,C2,ANTEL,0,C,C
34006632,12,41,1,7,0,user_unvalidated_image,voucher_voided,Android,otros,Primaria,Web,femenino,Android,RESTO,C1_3_3,ANTEL,1,C,C
37438759,10,7,1,2,0,idp_validated,eval_no_offer,Android,otros,Secundaria - Ciclo Básico,Facebook,femenino,Android,RESTO,C1_3_6,MOVISTAR,1,C,C
36127012,19,34,2,2,0,id_verified,voucher_issued,Android,emp_publico,Primaria,Facebook,masculino,Android,RESTO,C2,ANTEL,1,B1,B1
13295068,23,47,2,7,0,id_verified,voucher_issued,Android,jubilado,Secundaria - Bachillerato,Web,masculino,Android,CANELONES,C2,MOVISTAR,1,R,B1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39040629,11,50,1,4,1,id_verified,voucher_issued,Android,emp_publico,Secundaria - Ciclo Básico,Web,masculino,Android,RESTO,C1_2_3,ANTEL,1,B1,B1
33515436,12,41,1,7,0,id_verified,voucher_issued,Android,jubilado,Secundaria - Bachillerato,Web,masculino,Android,RESTO,C1_3_6,ANTEL,1,R,C
16411833,16,42,2,4,0,id_verified,voucher_issued,Android,jubilado,Educación Inicial,Web,masculino,Android,CANELONES,C1_3_1,MOVISTAR,1,M,C


##TR Características Nominales

In [342]:
df_cat.columns

Index(['o_Día', 'o_Semana_año', 'o_Quincena', 'o_Num_Dia_Sem', 'o_Zero',
       'o_status_usuario', 'o_status', 'o_sist_op_familia',
       'o_relacion_laboral', 'o_nivel_educativo', 'o_navegador_familia',
       'o_genero', 'o_dispositivo_marca', 'o_departamento', 'o_clr_accion',
       'o_cel_carrier', 'm_Contingencia', 'm_Grupo_b', 'm_Grupo_i'],
      dtype='object')

In [343]:
nominal_vars = ['o_Día', 'o_Semana_año', 'o_Quincena', 'o_Num_Dia_Sem', 'o_Zero','m_Contingencia','o_status_usuario','o_status', 'o_sist_op_familia','o_relacion_laboral', 'o_navegador_familia','o_genero','o_dispositivo_marca', 'o_departamento', 'o_cel_carrier','m_Grupo_b','m_Grupo_i']

# Identify nominal columns
nominal_cols = nominal_vars

# Apply One-Hot Encoding to nominal columns
df = pd.get_dummies(df, columns=nominal_cols)

# Display the first few rows of the dataframe with one-hot encoded variables
display(df.head())

# Display the information of the updated dataframe
#df.info()


,o_nivel_educativo,o_nivel_antiguedad_laboral,o_ingreso_declarado,o_edad,o_clr_score,o_clr_accion,m_Entidades,m_Exposición,m_Deuda,m_Rel_DeudaPrincipal,...,m_Grupo_b_B1,m_Grupo_b_C,m_Grupo_b_C1,m_Grupo_b_M,m_Grupo_b_R,m_Grupo_b_RR,m_Grupo_i_A,m_Grupo_i_B1,m_Grupo_i_C,m_Grupo_i_C1
m_Documento,,,,,,,,,,,,,,,,,,,,,
38753940,Primaria,4,9000,47,450.00,C2,4,81814,79923,0.44,...,False,True,False,False,False,False,False,False,True,False
34006632,Primaria,1,26000,46,581.00,C1_3_3,4,183144,155372,0.43,...,False,True,False,False,False,False,False,False,True,False
37438759,Secundaria - Ciclo Básico,1,40000,63,508.00,C1_3_6,8,490186,442449,0.33,...,False,True,False,False,False,False,False,False,True,False
36127012,Primaria,1,25000,53,666.00,C2,2,390907,325615,0.74,...,True,False,False,False,False,False,False,True,False,False
13295068,Secundaria - Bachillerato,1,31300,70,617.00,C2,1,108675,27514,0.25,...,False,False,False,False,True,False,False,True,False,False


##TR Características Ordinales

In [344]:
np.unique(df['o_nivel_educativo'])

array(['Educación Inicial', 'Educación Terciaria', 'Primaria',
       'Secundaria - Bachillerato', 'Secundaria - Ciclo Básico'],
      dtype=object)

In [345]:
edu_ord_map = {'Educación Inicial': 1, 'Primaria': 2, 'Secundaria - Ciclo Básico': 3,
               'Secundaria - Bachillerato': 4, 'Educación Terciaria': 5}

df['EduLabel'] = df['o_nivel_educativo'].map(edu_ord_map)
df[['o_nivel_educativo', 'EduLabel']].iloc[4:10]

,o_nivel_educativo,EduLabel
m_Documento,,
13295068,Secundaria - Bachillerato,4
42926002,Primaria,2
44758308,Educación Terciaria,5
33932466,Educación Terciaria,5
48280365,Secundaria - Bachillerato,4
36419621,Secundaria - Ciclo Básico,3


In [346]:
np.unique(df['o_clr_accion'])

array(['C1_2_3', 'C1_3_1', 'C1_3_3', 'C1_3_6', 'C2', 'C3', 'C4', 'Varios'],
      dtype=object)

In [347]:
edu_ord_map = {'C2': 8, 'C4': 7, 'C3': 6,
               'C1_2_3': 5, 'C1_3_1': 4, 'C1_3_3':3, 'C1_3_6':2, 'Varios':1}

df['CleLabel'] = df['o_clr_accion'].map(edu_ord_map)
df[['o_clr_accion', 'CleLabel']].iloc[4:10]

,o_clr_accion,CleLabel
m_Documento,,
13295068,C2,8
42926002,C3,6
44758308,C1_2_3,5
33932466,C3,6
48280365,C1_2_3,5
36419621,Varios,1


In [348]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1004 entries, 38753940 to 42719687
Columns: 142 entries, o_nivel_educativo to CleLabel
dtypes: bool(123), category(4), float64(5), int64(10)
memory usage: 251.1 KB


In [349]:
df['EduLabel'] = df['EduLabel'].astype(int)
df['CleLabel'] = df['CleLabel'].astype(int)

df = df.drop(['o_nivel_educativo', 'o_clr_accion'], axis=1)

#df.info()


##Datos numéricos

In [350]:
df_numeric = df.select_dtypes(include=np.number)
df_numeric.info()
display(df_numeric.head())

<class 'pandas.core.frame.DataFrame'>
Index: 1004 entries, 38753940 to 42719687
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   o_nivel_antiguedad_laboral  1004 non-null   int64  
 1   o_ingreso_declarado         1004 non-null   int64  
 2   o_edad                      1004 non-null   int64  
 3   o_clr_score                 1004 non-null   float64
 4   m_Entidades                 1004 non-null   int64  
 5   m_Exposición                1004 non-null   int64  
 6   m_Deuda                     1004 non-null   int64  
 7   m_Rel_DeudaPrincipal        1004 non-null   float64
 8   m_Var_dueda_3M              1004 non-null   float64
 9   m_Sin_Ficha                 1004 non-null   int64  
 10  m_Malos                     1004 non-null   int64  
 11  m_Mejora                    1004 non-null   int64  
 12  m_Q_irregular_12m           1004 non-null   int64  
 13  hora_sin                   

,o_nivel_antiguedad_laboral,o_ingreso_declarado,o_edad,o_clr_score,m_Entidades,m_Exposición,m_Deuda,m_Rel_DeudaPrincipal,m_Var_dueda_3M,m_Sin_Ficha,m_Malos,m_Mejora,m_Q_irregular_12m,hora_sin,hora_cos,EduLabel,CleLabel
m_Documento,,,,,,,,,,,,,,,,,
38753940,4,9000,47,450.00,4,81814,79923,0.44,0.92,0,0,0,0,-0.26,-0.97,2,8
34006632,1,26000,46,581.00,4,183144,155372,0.43,0.85,0,0,0,0,-0.87,0.50,2,3
37438759,1,40000,63,508.00,8,490186,442449,0.33,-0.04,0,0,0,0,-0.26,0.97,3,2
36127012,1,25000,53,666.00,2,390907,325615,0.74,-0.03,0,0,0,0,-0.50,0.87,2,8
13295068,1,31300,70,617.00,1,108675,27514,0.25,0.01,0,0,0,4,0.87,-0.50,4,8


In [351]:
df.describe()

,o_nivel_antiguedad_laboral,o_ingreso_declarado,o_edad,o_clr_score,m_Entidades,m_Exposición,m_Deuda,m_Rel_DeudaPrincipal,m_Var_dueda_3M,m_Sin_Ficha,m_Malos,m_Mejora,m_Q_irregular_12m,hora_sin,hora_cos,EduLabel,CleLabel
count,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00
mean,1.87,41640.74,45.63,635.58,3.29,358008.48,313631.82,0.59,0.75,0.27,0.15,0.28,1.06,-0.27,-0.26,3.33,4.76
std,1.08,38757.04,12.86,161.94,2.04,511646.72,472810.90,0.25,2.00,0.68,0.52,1.07,1.97,0.62,0.69,1.00,2.29
min,1.00,30.00,24.00,73.00,1.00,0.00,0.00,0.00,-0.76,0.00,0.00,-5.00,0.00,-1.00,-1.00,1.00,1.00
25%,1.00,23899.25,35.00,525.00,2.00,62074.75,50910.75,0.41,-0.02,0.00,0.00,0.00,0.00,-0.87,-0.87,3.00,3.00
50%,1.00,35000.00,45.00,644.00,3.00,177997.50,153643.50,0.57,0.08,0.00,0.00,0.00,0.00,-0.50,-0.50,3.00,5.00
75%,3.00,50000.00,55.00,759.25,4.00,464096.00,392158.25,0.79,0.50,0.00,0.00,0.00,1.00,0.26,0.26,4.00,7.00
max,4.00,560000.00,78.00,938.00,13.00,5587348.00,5365539.00,1.00,10.00,3.00,3.00,6.00,12.00,1.00,1.00,5.00,8.00


In [352]:
df_numeric.columns

Index(['o_nivel_antiguedad_laboral', 'o_ingreso_declarado', 'o_edad',
       'o_clr_score', 'm_Entidades', 'm_Exposición', 'm_Deuda',
       'm_Rel_DeudaPrincipal', 'm_Var_dueda_3M', 'm_Sin_Ficha', 'm_Malos',
       'm_Mejora', 'm_Q_irregular_12m', 'hora_sin', 'hora_cos', 'EduLabel',
       'CleLabel'],
      dtype='object')

In [353]:
# prompt: Preparar las variables numéricas, en función de la distribucíon de cada una, para aplicarlas en en modelo de aprendizaje automático, de manera que maximice el resultado

# Seleccionar las columnas numéricas para el escalado
numeric_cols_to_scale = ['o_edad', 'o_ingreso_mensual', 'o_antiguedad_laboral_meses',
                         'o_ingreso_estimado_ponderado', 'o_ingreso_nominal_ponderado',
                         'o_antiguedad_cliente_meses', 'o_clr_score',
                         'm_Patrimonio', 'm_Ingreso', 'm_Egreso', 'm_Antiguedad']

# Filter the list to include only columns present in the DataFrame
existing_numeric_cols_to_scale = [col for col in numeric_cols_to_scale if col in df.columns]

# Aplicar MinMaxScaler a las columnas numéricas
scaler = MinMaxScaler()
# Apply scaling only to the existing numeric columns
df[existing_numeric_cols_to_scale] = scaler.fit_transform(df[existing_numeric_cols_to_scale])

# Mostrar la información y las primeras filas del DataFrame con las variables numéricas escaladas
print("DataFrame después de escalar las variables numéricas:")
df_numeric_scaled = df.select_dtypes(include=np.number)
df_numeric_scaled.info()
display(df_numeric_scaled.head())


DataFrame después de escalar las variables numéricas:
<class 'pandas.core.frame.DataFrame'>
Index: 1004 entries, 38753940 to 42719687
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   o_nivel_antiguedad_laboral  1004 non-null   int64  
 1   o_ingreso_declarado         1004 non-null   int64  
 2   o_edad                      1004 non-null   float64
 3   o_clr_score                 1004 non-null   float64
 4   m_Entidades                 1004 non-null   int64  
 5   m_Exposición                1004 non-null   int64  
 6   m_Deuda                     1004 non-null   int64  
 7   m_Rel_DeudaPrincipal        1004 non-null   float64
 8   m_Var_dueda_3M              1004 non-null   float64
 9   m_Sin_Ficha                 1004 non-null   int64  
 10  m_Malos                     1004 non-null   int64  
 11  m_Mejora                    1004 non-null   int64  
 12  m_Q_irregular_12m           10

,o_nivel_antiguedad_laboral,o_ingreso_declarado,o_edad,o_clr_score,m_Entidades,m_Exposición,m_Deuda,m_Rel_DeudaPrincipal,m_Var_dueda_3M,m_Sin_Ficha,m_Malos,m_Mejora,m_Q_irregular_12m,hora_sin,hora_cos,EduLabel,CleLabel
m_Documento,,,,,,,,,,,,,,,,,
38753940,4,9000,0.43,0.44,4,81814,79923,0.44,0.92,0,0,0,0,-0.26,-0.97,2,8
34006632,1,26000,0.41,0.59,4,183144,155372,0.43,0.85,0,0,0,0,-0.87,0.50,2,3
37438759,1,40000,0.72,0.50,8,490186,442449,0.33,-0.04,0,0,0,0,-0.26,0.97,3,2
36127012,1,25000,0.54,0.69,2,390907,325615,0.74,-0.03,0,0,0,0,-0.50,0.87,2,8
13295068,1,31300,0.85,0.63,1,108675,27514,0.25,0.01,0,0,0,4,0.87,-0.50,4,8


In [330]:
df.describe()

,o_nivel_antiguedad_laboral,o_ingreso_declarado,o_edad,o_clr_score,m_Entidades,m_Exposición,m_Deuda,m_Rel_DeudaPrincipal,m_Var_dueda_3M,m_Sin_Ficha,m_Malos,m_Mejora,m_Q_irregular_12m,hora_sin,hora_cos,EduLabel,CleLabel
count,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00,1004.00
mean,1.87,41640.74,0.40,0.65,3.29,358008.48,313631.82,0.59,0.75,0.27,0.15,0.28,1.06,-0.27,-0.26,3.33,4.76
std,1.08,38757.04,0.24,0.19,2.04,511646.72,472810.90,0.25,2.00,0.68,0.52,1.07,1.97,0.62,0.69,1.00,2.29
min,1.00,30.00,0.00,0.00,1.00,0.00,0.00,0.00,-0.76,0.00,0.00,-5.00,0.00,-1.00,-1.00,1.00,1.00
25%,1.00,23899.25,0.20,0.52,2.00,62074.75,50910.75,0.41,-0.02,0.00,0.00,0.00,0.00,-0.87,-0.87,3.00,3.00
50%,1.00,35000.00,0.39,0.66,3.00,177997.50,153643.50,0.57,0.08,0.00,0.00,0.00,0.00,-0.50,-0.50,3.00,5.00
75%,3.00,50000.00,0.57,0.79,4.00,464096.00,392158.25,0.79,0.50,0.00,0.00,0.00,1.00,0.26,0.26,4.00,7.00
max,4.00,560000.00,1.00,1.00,13.00,5587348.00,5365539.00,1.00,10.00,3.00,3.00,6.00,12.00,1.00,1.00,5.00,8.00


In [355]:
df_numeric.columns

Index(['o_nivel_antiguedad_laboral', 'o_ingreso_declarado', 'o_edad',
       'o_clr_score', 'm_Entidades', 'm_Exposición', 'm_Deuda',
       'm_Rel_DeudaPrincipal', 'm_Var_dueda_3M', 'm_Sin_Ficha', 'm_Malos',
       'm_Mejora', 'm_Q_irregular_12m', 'hora_sin', 'hora_cos', 'EduLabel',
       'CleLabel'],
      dtype='object')

In [356]:
# Seleccionar las variables numéricas que se discretizarán por cuantiles
vars_to_discretize_quantiles = ['o_ingreso_declarado', 'm_Entidades',
                                 'm_Exposición', 'm_Deuda']

# Discretizar cada variable seleccionada por cuantiles
for col in vars_to_discretize_quantiles:
  if col in df.columns:
    # Crear 4 bins (quintiles) basados en los cuantiles
    df[col + '_quantile_bin'] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
    # Convertir la nueva columna discretizada a tipo 'category'
    df[col + '_quantile_bin'] = df[col + '_quantile_bin'].astype('category')
  else:
    print(f"Warning: Column '{col}' not found in DataFrame for quantile discretization.")

# Mostrar las primeras filas del DataFrame con las nuevas variables discretizadas
print("\nDataFrame después de discretizar por cuantiles:")
display(df.head())

# Mostrar la información del DataFrame para ver los tipos de datos de las nuevas columnas
df.info()


DataFrame después de discretizar por cuantiles:


,m_Documento,o_nivel_antiguedad_laboral,o_ingreso_declarado,o_edad,o_clr_score,m_Entidades,m_Exposición,m_Deuda,m_Rel_DeudaPrincipal,m_Var_dueda_3M,...,m_Grupo_i_B1,m_Grupo_i_C,m_Grupo_i_C1,EduLabel,CleLabel,Edad_bin_round,o_ingreso_declarado_quantile_bin,m_Entidades_quantile_bin,m_Exposición_quantile_bin,m_Deuda_quantile_bin
0,38753940,4,9000,0.43,0.44,4,81814,79923,0.44,0.92,...,False,True,False,2,8,0.00,0,2,1,1
1,34006632,1,26000,0.41,0.59,4,183144,155372,0.43,0.85,...,False,True,False,2,3,0.00,1,2,2,2
2,37438759,1,40000,0.72,0.50,8,490186,442449,0.33,-0.04,...,False,True,False,3,2,0.00,3,3,3,3
3,36127012,1,25000,0.54,0.69,2,390907,325615,0.74,-0.03,...,True,False,False,2,8,0.00,1,0,3,3
4,13295068,1,31300,0.85,0.63,1,108675,27514,0.25,0.01,...,True,False,False,4,8,0.00,2,0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Columns: 146 entries, m_Documento to m_Deuda_quantile_bin
dtypes: bool(123), category(4), float64(7), int64(12)
memory usage: 274.5 KB
